# Iteratorパターン

* 

In [2]:
%%bash
### 変数の設定
name=Aggregate

### ソースの編集
cat <<- EOS > ${name}.java
public interface ${name} {
    public abstract Iterator iterator();
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

In [3]:
%%bash
### 変数の設定
name=Iterator

### ソースの編集
cat <<- EOS > ${name}.java
public interface ${name} {
    public abstract boolean hasNext();
    public abstract Object next();
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

In [4]:
%%bash
### 変数の設定
name=Book

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} {
    private String name;
    public Book(String name) {
        this.name = name;
    }
    public String getName() {
        return name;
    }
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

In [13]:
%%bash
### 変数の設定
name=BookShelf

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} implements Aggregate {
    private Book[] books;
    private int last = 0;
    public BookShelf(int maxsize) {
        this.books = new Book[maxsize];
    }
    public Book getBookAt(int index) {
        return books[index];
    }
    public void appendBook(Book book) {
        this.books[last] = book;
        last++;
    }
    public int getLength() {
        return last;
    }
    public Iterator iterator() {
        return new BookShelfIterator(this);
    }
}
EOS

### コンパイル
#javac ${name}.java
#javac -encoding UTF-8 ${name}.java

In [24]:
%%bash
### 変数の設定
name=BookShelfIterator

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} implements Iterator {
    private BookShelf bookShelf;
    private int index;
    public BookShelfIterator(BookShelf bookShelf) {
        this.bookShelf = bookShelf;
        this.index = 0;
    }
    public boolean hasNext() {
        if (index < bookShelf.getLength()) {
            return true;
        } else {
            return false;
        }
    }
    public Object next() {
        Book book = bookShelf.getBookAt(index);
        index++;
        return book;
    }
}
EOS

### コンパイル
#javac ${name}.java
javac -encoding UTF-8 ${name}.java

In [26]:
%%bash
### 変数の設定
name=Main

### ソースの編集
cat <<- EOS > ${name}.java
public class ${name} {
    public static void main(String[] args) {
        BookShelf bookShelf = new BookShelf(4);
        bookShelf.appendBook(new Book("Around the World in 80 Days"));
        bookShelf.appendBook(new Book("Bible"));
        bookShelf.appendBook(new Book("Cinderella"));
        bookShelf.appendBook(new Book("Daddy-Long-Legs"));
        Iterator it = bookShelf.iterator();
        while (it.hasNext()) {
            Book book = (Book) it.next();
            System.out.println(book.getName());
        }
    }
}
EOS

### コンパイル
javac ${name}.java
#javac -encoding UTF-8 ${name}.java

### 実行
#java -classpath ${classpath} ${name}
java ${name}

Around the World in 80 Days
Bible
Cinderella
Daddy-Long-Legs
